In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
import dlib
import os
import pickle
np.random.seed(1000)


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
frames = []
labels = []
Engagment = []
confusion = []
frustration = []
bordam = []
temp = []
for file in os.listdir('output_engage/'):
    if file[-10:] == 'frames.pkl':
        with open('output_engage/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        temp = []
        with open('output_engage/'+file, 'rb') as f:
            Engagment = pickle.load(f)
        with open('output_confusion/'+file, 'rb') as fc:
            confusion = pickle.load(fc)
        with open('output_frustration/'+file, 'rb') as ff:
            frustration = pickle.load(ff)
        with open('output_bordam/'+file, 'rb') as fb:
            bordam = pickle.load(fb)
        temp.extend(Engagment)
        temp.extend(confusion)
        temp.extend(frustration)
        temp.extend(bordam)
        labels.append(temp)

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [[label[0], label[299], label[599], label[899]]] * len(clip)
    except Exception as e:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [[label[0], label[299], label[599], label[899]]] * len(clip)
    except Exception as e:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [7]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [8]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [9]:
train_labels //= 2
test_labels //= 2

In [25]:
print(train_labels[0])

[1 1 0 0]


In [26]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=288, input_shape=(48,48,1), kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=288, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=288, kernel_size=3, strides=2, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=144, kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.MaxPooling2D(pool_size=3, strides=2),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=4, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(4, activation='sigmoid')
])

In [27]:
model.compile(optimizer='adamax',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(train_labels, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

{0: 0.25383174022333826, 1: 1631.2709731543623, 2: 26.56386612021858, 3: 45.19932589493259}


In [32]:
model.fit(train_images, train_labels, class_weight = d_class_weights, epochs=5, batch_size=50)

  ...
    to  
  ['...']
Train on 1944475 samples
Epoch 1/5
1944475/1944475 [==============================] - 1986s 1ms/sample - loss: 1.5320 - accuracy: 0.5748
Epoch 2/5
1944475/1944475 [==============================] - 1977s 1ms/sample - loss: 1.5056 - accuracy: 0.5350
Epoch 3/5
1944475/1944475 [==============================] - 1977s 1ms/sample - loss: 1.4888 - accuracy: 0.5015
Epoch 4/5
1944475/1944475 [==============================] - 1978s 1ms/sample - loss: 1.4756 - accuracy: 0.5124
Epoch 5/5
1944475/1944475 [==============================] - 1976s 1ms/sample - loss: 1.4665 - accuracy: 0.4936


In [33]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

482212/482212 [==============================] - 157s 326us/sample - loss: 2.3008 - accuracy: 0.4715


In [34]:
model.save('allconv8labels.h5')